In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pathlib
from typing import Type
import json

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
import equinox as eqx

from mc2.runners.model_setup_jax import get_normalizer
from mc2.data_management import EXPERIMENT_LOGS_ROOT, MODEL_DUMP_ROOT, NORMALIZATION_ROOT, load_hdf5_pretest_data
from mc2.models.jiles_atherton import JAStatic, JAWithGRU
from mc2.models.RNN import GRU
from mc2.models.model_interface import ModelInterface, load_model, RNNwInterface, JAwInterface

In [ ]:
from mc2.utils.model_evaluation import reconstruct_model_from_exp_id, get_exp_ids

In [ ]:
get_exp_ids(material_name=["N87"])

In [ ]:
exp_id = '3C90_GRU_6dca3c2e-c8a0-47'
material_name = exp_id.split("_")[0]
model_type = exp_id.split("_")[1]

seed = 0

In [ ]:
wrapped_model = reconstruct_model_from_exp_id(exp_id)
wrapped_model

## Look at stored predictions + losses:

In [ ]:
from mc2.utils.model_evaluation import (
    load_gt_and_pred, plot_worst_predictions, plot_first_predictions, plot_loss_trends
)

In [ ]:
gt, pred = load_gt_and_pred(
    exp_id=exp_id,
    seed=seed,
    freq_idx=0
)

In [ ]:
plot_worst_predictions(gt, pred);

In [ ]:
plot_first_predictions(gt, pred);

In [ ]:
plot_loss_trends(exp_id, seed);

## Loading and evaluating stored models:

In [ ]:
wrapped_model = reconstruct_model_from_exp_id(exp_id)
wrapped_model

### Plotting:

In [ ]:
from mc2.training.jax_routine import val_test
from mc2.runners.model_setup_jax import get_normalizer

In [ ]:
_, (train_set, eval_set, test_set) = get_normalizer(material_name, wrapped_model.featurize, subsampling_freq=1, do_normalization=True)

In [ ]:
test_loss, test_pred_l, test_gt_l = val_test(test_set, wrapped_model, past_size=1)

In [ ]:
# plot_worst_predictions(gt, pred)  # to compare to the performance at the end of training
plot_worst_predictions(test_gt_l[0], test_pred_l[0])

### Parameter counting:

**Q:** Assumes that all parameters are `jax.Arrays`. Is this valid? Essentially the same as `.requires_grad()`

In [ ]:
from mc2.models.model_interface import count_model_parameters

In [ ]:
count_model_parameters(wrapped_model)

In [ ]:
# test_model, _, params, data_tuple = setup_model(
#     model_label = "GRU",
#     material_name=material_name,
#     model_key=jax.random.PRNGKey(0),
# )
# count_model_parameters(test_model)

# test_model, _, params, data_tuple = setup_model(
#     model_label = "JAWithExternGRU",
#     material_name=material_name,
#     model_key=jax.random.PRNGKey(0),
# )

## Pretest eval:

In [ ]:
from IPython.display import display, HTML
from mc2.utils.pretest_evaluation import evaluate_pretest_scenarios_custom, create_multilevel_df, HOSTS_VALUES_DICT, SCENARIO_LABELS

In [ ]:
B, T, H_init, H_true, loss, loss_short, msks_scenarios_N_tup = load_hdf5_pretest_data(material_name)
B.shape, T.shape, H_init.shape, H_true.shape

In [ ]:
metrics = evaluate_pretest_scenarios_custom(
    wrapped_model,
    B,
    T,
    H_init,
    H_true,
    loss,
    msks_scenarios_N_tup,
    scenario_labels=SCENARIO_LABELS,
    show_plots=False,
)

In [ ]:
df_models_3C90 = create_multilevel_df({
    "hosts": HOSTS_VALUES_DICT["3C90"],
    "GRU": metrics}
)
display(HTML(df_models_3C90.T.to_html(float_format="%.3f", bold_rows=False)))

In [ ]:
df_models_3C90 = create_multilevel_df({
    "hosts": HOSTS_VALUES_DICT["N87"],
    "GRU": metrics}
)
display(HTML(df_models_3C90.T.to_html(float_format="%.3f", bold_rows=False)))